In [ ]:
from fastai.tabular import *
import pandas as pd
import numpy as np
from sklearn import model_selection
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, GridSearchCV,KFold, cross_val_score
import torch
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from scipy.sparse import hstack
from tqdm import tqdm
from sklearn.utils.multiclass import unique_labels

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_path = '/content/drive/MyDrive/[졸프]데이터_csv/244375-data_1010.csv'
data = pd.read_csv(data_path)

In [ ]:
data

,url,label,url_length,character_count,https,ttl,name_server,IP/email,short_url,file_extension,ns_domain_match
0,https://montpelier.org,0,22,3,1,600.0,"['ns02.domaincontrol.com', 'ns01.domaincontrol...",0,0,0,0.0
1,https://notion.com,0,18,3,1,300.0,"['woz.ns.cloudflare.com', 'dana.ns.cloudflare....",0,0,0,0.0
2,https://icanvas.com,0,19,3,1,86400.0,"['pdns01.domaincontrol.com', 'pdns02.domaincon...",0,0,0,0.0
3,https://alloy.com,0,17,3,1,300.0,"['jack.ns.cloudflare.com', 'nola.ns.cloudflare...",0,0,0,0.0
4,https://ynhhs.org,0,17,3,1,300.0,"['ns31.constellix.com', 'ns51.constellix.net',...",0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
172407,https://bitbucket.org/foxxlrep/repo/downloads/...,1,53,8,1,60.0,"['ns-1305.awsdns-35.org', 'ns-1746.awsdns-26.c...",0,0,1,0.0
172408,https://github.com/delta-io/delta/files/150161...,1,58,10,1,58.0,"['ns-1283.awsdns-32.org', 'ns-421.awsdns-52.co...",0,0,1,0.0
172409,https://uploaddeimagens.com.br/images/004/801/...,1,72,11,1,300.0,"['iris.ns.cloudflare.com', 'phil.ns.cloudflare...",0,0,1,0.0
172410,https://mussangroup.com/wp-content/images/pic6...,1,50,8,1,14293.0,"['ns1.veridyen.com', 'ns2.veridyen.com']",0,0,1,0.0


### **데이터 전처리**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

malicious_path = "/content/drive/MyDrive/gra_project/졸업프로젝트/[졸프]데이터_csv/malicous.csv"
benign_path="/content/drive/MyDrive/gra_project/졸업프로젝트/[졸프]데이터_csv/benign.csv"

benign_data = pd.read_csv(benign_path)
malicious_data = pd.read_csv(malicious_path)

benign_data = benign_data.iloc[:, [0]]
malicious_data = malicious_data.iloc[:, [0]]


benign_data.columns=['url']
malicious_data.columns=['url']

benign_data['label'] = 0
malicious_data['label'] = 1

# url 열에서 중복되는 값 제거 (첫 번째만 남기고 나머지 제거)
benign_data_clean = benign_data.drop_duplicates(subset=['url'])
malicious_data_clean = malicious_data.drop_duplicates(subset=['url'])


data=pd.concat([benign_data,malicious_data],ignore_index=True)




In [ ]:
print(len(benign_data_clean))
print(len(malicious_data_clean))
print(len(malicious_data_clean)+len(benign_data_clean))

149707
94067
243774


In [ ]:
# # 테스트용 미니 데이터

# mini_benign = benign_data[:20:]
# mini_malicious = malicious_data[:30:]
# data_mini=pd.concat([mini_benign,mini_malicious],ignore_index=True)

In [ ]:
# data_mini


,url,label
0,https://facebook.com,0
1,https://youtube.com,0
2,https://twitter.com,0
3,https://instagram.com,0
4,https://linkedin.com,0
5,https://apple.com,0
6,googletagmanager.com,0
7,https://microsoft.com,0
8,https://wikipedia.org,0
9,wordpress.org,0


In [ ]:
# URL의 길이
def length_url(url):
    return len(url)


In [ ]:
### 특수 문자 개수 카운트
# 특수 문자 리스트
special_chars = ['@', '?', '-', '=', '.', '#', '%', '+', '$', '!', '*', ',', '/']

# 특수 문자의 개수를 특징으로 추가
def count_special_characters(url):
    return sum(url.count(char) for char in special_chars)

In [ ]:
# HTTPS 존재 여부
def https_exist(url):
    return 1 if url.startswith('https://') else 0
# df['https'] = df['url'].apply(lambda x: 1 if x.startswith('https://') else 0)

In [ ]:
# #TTL 가져오기
# from urllib.parse import urlparse

# def get_ttl(domain):
#     try:
#         # URL에서 도메인 이름만 추출
#         parsed_url = urlparse(domain)
#         domain_name = parsed_url.netloc if parsed_url.netloc else parsed_url.path

#         # DNS 쿼리 수행
#         answers = dns.resolver.resolve(domain_name, 'SOA')
#         for rdata in answers:
#             return rdata.minimum
#     except Exception as e:
#         print(f"Error resolving TTL for {domain_name}: {e}")
#         return None

In [ ]:
import dns.resolver
from urllib.parse import urlparse

def get_ttl(domain):
    try:
        # URL에서 도메인 이름만 추출
        parsed_url = urlparse(domain)
        domain_name = parsed_url.netloc if parsed_url.netloc else parsed_url.path

        # A 레코드로 TTL 정보 조회
        answers = dns.resolver.resolve(domain_name, 'A')

        # 첫 번째 A 레코드의 TTL 값 반환
        ttl = answers.ttl
        return ttl
    except Exception as e:
        print(f"Error resolving TTL for {domain_name}: {e}")
        return None

In [ ]:
# name server 가져오기
def get_ns(domain):
    try:

      # URL에서 도메인 이름만 추출
        parsed_url = urlparse(domain)
        domain_name = parsed_url.netloc if parsed_url.netloc else parsed_url.path

        answers = dns.resolver.resolve(domain_name, 'NS')
        ns_records = [str(rdata.target).rstrip('.') for rdata in answers]
        return ns_records
    except Exception as e:
        #print(f"Error resolving NS for {domain_name}: {e}")
        return []

In [ ]:
'''
# ip 주소 가져오기
def get_ip(domain):
    try:
      # URL에서 도메인 이름만 추출
        parsed_url = urlparse(domain)
        domain_name = parsed_url.netloc if parsed_url.netloc else parsed_url.path

        answers = dns.resolver.resolve(domain_name, 'A')
        return [rdata.address for rdata in answers]
    except Exception as e:
        print(f"Error resolving IP for {domain_name}: {e}")
        return []
  '''

'\n# ip 주소 가져오기\ndef get_ip(domain):\n    try:\n      # URL에서 도메인 이름만 추출\n        parsed_url = urlparse(domain)\n        domain_name = parsed_url.netloc if parsed_url.netloc else parsed_url.path\n\n        answers = dns.resolver.resolve(domain_name, \'A\')\n        return [rdata.address for rdata in answers]\n    except Exception as e:\n        print(f"Error resolving IP for {domain_name}: {e}")\n        return []\n  '

In [ ]:
'''
# net name 가져오기

def get_netname(ip_address):
    try:
        whois_info = whois.whois(ip_address)
        return whois_info.get('netname', 'Unknown')
    except Exception as e:
        print(f"Error resolving Netname for {ip_address}: {e}")
        return 'Unknown'
  '''

'\n# net name 가져오기\n\ndef get_netname(ip_address):\n    try:\n        whois_info = whois.whois(ip_address)\n        return whois_info.get(\'netname\', \'Unknown\')\n    except Exception as e:\n        print(f"Error resolving Netname for {ip_address}: {e}")\n        return \'Unknown\'\n  '

In [ ]:
def group_and_compare_netnames(netnames):
    from collections import Counter
    netname_counts = Counter(netnames)
    suspicious_netnames = {netname for netname, count in netname_counts.items() if count > 1}
    return suspicious_netnames

In [ ]:
# IP 혹은 이메일이 URL에 포함되어 있는 경우 악성으로 판단
def ip_email_exist(url):
    # IP 주소 패턴 (IPv4)
    ip_pattern = r'(?:\d{1,3}\.){3}\d{1,3}'

    # 이메일 주소 패턴
    email_pattern = r'[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+'

    # IP 주소가 포함된 경우
    if re.search(ip_pattern, url):
        return 1

    # 이메일 주소가 포함된 경우
    if re.search(email_pattern, url):
        return 1

    # 둘 다 해당되지 않으면 피싱 URL 아님
    return 0

In [ ]:
# 단축 url
def is_short_url(url):
    # 단축 URL 패턴 (여기서는 일부만 포함, 필요한 만큼 추가 가능)
    short_url_pattern = r'bit\.ly|goo\.gl|t\.co|tinyurl\.com|me2\.kr|is\.gd|shrtco\.de|ow\.ly|buff\.ly|mcaf\.ee|surl\.kr'

    # 단축 URL 도메인이 포함되어 있는지 확인
    if re.search(short_url_pattern, url):
        return 1  # 단축 URL이 포함된 경우
    else:
        return 0  # 포함되지 않은 경우


In [ ]:
def file_extension(url):
    # 파일 확장자 패턴
    file_extension_pattern = r'\.apk|\.pdf|\.jpg|\.png|\.zip|\.html|\.docx|\.xlsx|\.pptx'

    # URL에 파일 확장자가 포함되어 있는지 확인
    if re.search(file_extension_pattern, url, re.IGNORECASE):
        return 1  # 확장자 포함
    else:
        return 0  # 확장자 불포함

In [ ]:
# 'character_count' 열 추가
data['url_length'] = data['url'].apply(length_url)
data['character_count'] = data['url'].apply(count_special_characters)
data['https'] = data['url'].apply(https_exist)

In [ ]:


# tqdm을 pandas에 확장
tqdm.pandas()

# get_ttl 및 get_ns 함수를 data['url']에 적용하면서 진행 상황을 표시
data['ttl'] = data['url'].progress_apply(get_ttl)



Output hidden; open in https://colab.research.google.com to view.

In [ ]:
data['name_server'] = data['url'].progress_apply(get_ns)

100%|██████████| 30000/30000 [53:34<00:00,  9.33it/s]


In [ ]:

data['IP/email'] = data['url'].apply(ip_email_exist)
data['short_url'] = data['url'].apply(is_short_url)
data['file_extension'] = data['url'].apply(file_extension)

In [ ]:
data

,url,label,url_length,character_count,https,ttl,name_server,IP/email,short_url,file_extension
0,https://facebook.com,0,20,2,1,300.0,"[c.ns.facebook.com, a.ns.facebook.com, b.ns.fa...",0,0,0
1,https://youtube.com,0,19,2,1,60.0,"[ns1.google.com, ns2.google.com, ns4.google.co...",0,0,0
2,https://twitter.com,0,19,2,1,3600.0,"[b.r06.twtrdns.net, a.r06.twtrdns.net, c.r06.t...",0,0,0
3,https://instagram.com,0,21,2,1,3600.0,"[c.ns.instagram.com, b.ns.instagram.com, a.ns....",0,0,0
4,https://linkedin.com,0,20,2,1,300.0,"[dns1.p09.nsone.net, dns2.p09.nsone.net, dns3....",0,0,0
...,...,...,...,...,...,...,...,...,...,...
95662,http://ip193.ip-146-59-161.eu:7358/GE-en.zip,1,44,8,0,NaN,[],0,0,1
95663,http://91.92.243.32/kvro/venture07.png,1,38,5,0,NaN,[],1,0,1
95664,http://avastcv.com/Avastavv.apk,1,31,3,0,NaN,[],0,0,1
95665,https://raw.githubusercontent.com/mmdfgg/2023/...,1,92,7,1,NaN,[],0,1,1


In [ ]:
"""def ns_domain_match(row):
    if not row['name_server']:
        return None
    url_domain_parts = row['url'].split('.')
    for ns_domain in row['name_server']:
        ns_domain_parts = ns_domain.split('.')
        if len(url_domain_parts) > 1 and len(ns_domain_parts) > 1:
            if url_domain_parts[-2:] == ns_domain_parts[-2:]:
                return 1
    return 0
data_mini['ns_domain_match'] = data_mini.apply(ns_domain_match, axis=1)"""


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def cosine_similarity_domains(url, name_servers):
    # URL 벡터화
    vectorizer = TfidfVectorizer()
    # 네임 서버를 하나의 문자열로 변환
    name_servers_str = ' '.join(name_servers) if name_servers else ''
    # 두 도메인을 포함한 리스트
    domains = [url, name_servers_str]

    # TF-IDF 행렬 생성
    tfidf_matrix = vectorizer.fit_transform(domains)

    # 코사인 유사도 계산
    similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])

    return similarity[0][0]

def ns_domain_match_cosine(row):
    if not row['name_server']:
        return None
    url = row['url']
    name_servers = row['name_server']

    # 코사인 유사도 계산
    similarity_score = cosine_similarity_domains(url, name_servers)

    # 유사도가 0.5 이상이면 일치로 판단 (임의 기준)
    return 1 if similarity_score >= 0.5 else 0



In [ ]:
#data_mini['netname'] = data_mini['url'].apply(lambda url: get_netname(get_ip(url)[0]) if get_ip(url) else 'Unknown')

In [ ]:
# 새로운 열 추가
data['ns_domain_match'] = data.progress_apply(ns_domain_match_cosine, axis=1)

100%|██████████| 13465/13465 [00:36<00:00, 371.31it/s]


,url,label,url_length,character_count,https,ttl,name_server,IP/email,short_url,file_extension,ns_domain_match
0,https://montpelier.org,0,22,2,1,600.0,"[ns01.domaincontrol.com, ns02.domaincontrol.com]",0,0,0,0.0
1,https://notion.com,0,18,2,1,1800.0,"[dana.ns.cloudflare.com, woz.ns.cloudflare.com]",0,0,0,0.0
2,https://icanvas.com,0,19,2,1,600.0,"[pdns01.domaincontrol.com, pdns02.domaincontro...",0,0,0,0.0
3,https://alloy.com,0,17,2,1,1800.0,"[jack.ns.cloudflare.com, nola.ns.cloudflare.com]",0,0,0,0.0
4,https://ynhhs.org,0,17,2,1,180.0,"[ns51.constellix.net, ns61.constellix.net, ns1...",0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
13460,https://bit.ly/3XWVOWv,1,22,2,1,86400.0,"[ns-1372.awsdns-43.org, ns-1766.awsdns-28.co.u...",0,1,0,0.0
13461,https://bit.ly/4cVkp29,1,22,2,1,86400.0,"[ns-1372.awsdns-43.org, ns-1766.awsdns-28.co.u...",0,1,0,0.0
13462,https://bit.ly/3zQPpSv,1,22,2,1,86400.0,"[ns-1766.awsdns-28.co.uk, ns-1372.awsdns-43.or...",0,1,0,0.0
13463,https://bit.ly/3Y16nHU,1,22,2,1,86400.0,"[ns-1372.awsdns-43.org, ns-cloud-c1.googledoma...",0,1,0,0.0


In [ ]:
data

In [ ]:
#수집한 데이터 저장
data.to_csv('data_1010.csv', index=False, encoding='utf-8')

###drop 하지 않은 data

In [ ]:
X = data[['url', 'url_length','character_count','https','ttl', 'ns_domain_match', 'IP/email', 'short_url','file_extension']]
y = data['label']

In [ ]:
vectorizer = TfidfVectorizer()
url_features = vectorizer.fit_transform(X['url'])
additional_features = X[['ttl']].values
#additional_features = SimpleImputer(strategy='mean').fit_transform(additional_features)
scaler = StandardScaler()
additional_features = scaler.fit_transform(additional_features)
#X_combined = hstack([url_features, additional_features, netname_features])
X_combined = hstack([url_features, additional_features])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2,random_state=42)

In [ ]:
model = LogisticRegression(max_iter=1000, solver='lbfgs', class_weight='balanced')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print('Accuracy:', accuracy_score(y_test, y_pred))
print('Classification Report:\n', classification_report(y_test, y_pred, zero_division=1))


Accuracy: 0.9881680828234203
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.99      0.99     28408
           1       0.95      0.98      0.97      6075

    accuracy                           0.99     34483
   macro avg       0.97      0.99      0.98     34483
weighted avg       0.99      0.99      0.99     34483



In [ ]:
# function for comparing models' results, accuracies, classifiers
def runExperiments(X_train, Y_train):
    random_seed = 42
    scoring = 'accuracy'
    models = []


    # List of models with hyperparameters to tune
    models.append(('LR', LogisticRegression(max_iter=1000, solver='lbfgs', class_weight='balanced')))
    models.append(('LDA', LinearDiscriminantAnalysis()))
    models.append(('KNN', KNeighborsClassifier()))
    models.append(('CART', DecisionTreeClassifier()))
    models.append(('NB', GaussianNB()))
    models.append(('SVM', SVC(gamma='auto')))
    models.append(('AB', AdaBoostClassifier(n_estimators=100)))
    models.append(('RF', RandomForestClassifier(n_estimators=10)))

    # Hyperparameters for tuning
    param_grid = {
        'LR': {'C': [0.01, 0.1, 1, 10]},
        'KNN': {'n_neighbors': [3, 5, 7]},
        'CART': {'max_depth': [None, 5, 10, 15]},
        'SVM': {'C': [0.01, 0.1, 1, 10], 'gamma': ['scale', 'auto']},
        'RF': {'n_estimators': [10, 50, 100], 'max_depth': [None, 5, 10, 15]}
    }

    # evaluate each model in turn using 10-fold cross-validation
    results = []
    accuracies = []

    for classifier, model in models:
        print(f"Evaluating {classifier}...")

        # Hyperparameter tuning using GridSearchCV if applicable
        if classifier in param_grid:
            grid = GridSearchCV(model, param_grid[classifier], scoring=scoring, cv=10)
            grid.fit(X_train, Y_train)
            best_model = grid.best_estimator_
            best_params = grid.best_params_
            print(f"Best parameters for {classifier}: {best_params}")
            y_pred = best_model.predict(X_train)  # Prediction on training set for evaluation
        else:
            kfold = model_selection.KFold(n_splits=10, shuffle=True, random_state=random_seed)
            cv_results = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
            results.append(cv_results)
            accuracies.append((cv_results.mean(), classifier))
            continue

        # Calculate performance metrics
        accuracy = accuracy_score(Y_train, y_pred)
        precision = precision_score(Y_train, y_pred, average='weighted', zero_division=1)
        recall = recall_score(Y_train, y_pred, average='weighted', zero_division=1)
        f1 = f1_score(Y_train, y_pred, average='weighted', zero_division=1)

        accuracies.append((accuracy, classifier))
        results.append(cv_results)  # Append results from cross-validation (can be averaged)

        msg = (f"{classifier}: Accuracy: {accuracy:.4f}, "
               f"Precision: {precision:.4f}, "
               f"Recall: {recall:.4f}, "
               f"F1 Score: {f1:.4f}")
        print(msg)

    return results, accuracies


In [ ]:
# Usage example
results, accuracies = runExperiments(X_train, y_train)

Evaluating LR...
Best parameters for LR: {'C': 10}


UnboundLocalError: local variable 'cv_results' referenced before assignment